# 📊 Proyecto de Ciencia de Datos para Empresa de Suscripciones  
## Predicción de Fuga de Clientes (Customer Churn) y Priorización de Retención

---

## 1️⃣ Contexto de negocio y motivación estratégica

Este proyecto representa el caso de una **empresa real de servicios por suscripción** (por ejemplo, telecom / SaaS B2C/B2B) que:

- Ofrece planes **mensuales** y **anuales**.
- Comercializa varios servicios: internet, telefonía, servicios adicionales.
- Factura de forma **recurrente** por cliente, generando un **MRR (Monthly Recurring Revenue)** estable.

La dirección detectó problemas claros:

- La **tasa de churn** (clientes que cancelan el servicio) ha ido en aumento.
- No existe un mecanismo **predictivo** para anticipar qué clientes están en riesgo.
- Las campañas de retención se ejecutan de forma **masiva** y poco enfocada:
  - Alto costo en descuentos y llamadas.
  - Bajo impacto real en reducción de fuga.

A nivel financiero, esto se traduce en:

- Menor **LTV (Lifetime Value)** de los clientes.
- Mayor presión en **Adquisición de Clientes (CAC)** para compensar los que se van.
- Dificultad para planear crecimiento y capacidad operativa.

---

### Pregunta de negocio

> **¿Podemos construir un modelo de riesgo de churn que permita priorizar a qué clientes contactar y qué segmentos requieren acciones estructurales de mejora del servicio?**

---

### Objetivos de negocio

1. **Cuantificar el churn actual** y entender sus drivers principales.
2. Entrenar un **modelo de clasificación** que estime la probabilidad de churn por cliente.
3. Generar una **lista priorizada** de clientes de alto riesgo para campañas de retención.
4. Traducir los resultados a **recomendaciones accionables** para:
   - Customer Success y Retención.
   - Marketing (ofertas, campañas).
   - Operaciones y Experiencia de Cliente (mejora de servicio).

---

### Métricas clave para evaluar la solución

- **F1-score de la clase churn**: equilibrio entre precisión y recall.
- **Recall de la clase churn**: qué proporción de clientes que realmente se van logramos identificar.
- **ROC-AUC**: capacidad del modelo para separar churn vs no churn.
- **Lift en el top 10–20%** de clientes ordenados por riesgo:
  - ¿Qué tanto aumenta la tasa de churn en el segmento priorizado vs la tasa global?

---

> Este notebook está diseñado para ser presentado a un **director de Retención/Customer Success** o a la **Dirección de Datos**, mostrando un flujo completo de trabajo con foco en impacto de negocio y no solo en métricas técnicas.


In [ ]:
# 1️⃣ Código – Imports y configuración global

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix
)

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Configuración estética
pd.set_option("display.max_columns", 100)
pd.set_option("display.float_format", lambda x: f"{x:,.3f}")
sns.set(style="whitegrid")
np.random.seed(42)

print("✅ Entorno preparado. Listo para cargar datos.")


## 2️⃣ Carga del dataset desde Excel y perfilado inicial

Trabajaremos con un dataset de producción exportado a Excel:

> `churn_clientes_sintetico_40k.xlsx`

Este dataset representa el histórico reciente de clientes de la empresa, con columnas como:

- Identificador y antigüedad (`customer_id`, `tenure_months`).
- Segmento y tipo de contrato (`segment`, `contract_type`).
- Servicios contratados (`has_internet`, `has_phone`, `num_services`).
- Facturación (`monthly_charges`, `total_charges`).
- Soporte y pagos (`num_tickets_6m`, `late_payments_6m`, `has_discount`).
- Método de pago (`payment_method`).
- Variable objetivo (`churn` = 1 si el cliente abandonó el servicio).

En este punto:

- Verificamos tamaño del dataset.
- Revisamos tipos de datos.
- Cuantificamos valores faltantes.
- Medimos la tasa actual de churn.

Esto ya da valor al negocio porque responde:  
**“¿De qué tamaño es el problema y qué tan balanceada está la base?”**


In [ ]:
# 2️⃣ Código – Carga de datos y chequeos básicos

FILE_PATH = "churn_clientes_sintetico_40k.xlsx"

try:
    df = pd.read_excel(FILE_PATH)
except FileNotFoundError:
    raise FileNotFoundError(
        f"No se encontró el archivo '{FILE_PATH}'. "
        "Asegúrate de que esté en la misma carpeta que este notebook."
    )

n_rows, n_cols = df.shape

print(f"📂 Dataset cargado correctamente: {n_rows:,} filas, {n_cols} columnas\n")

print("🔎 Vista rápida de las primeras filas:")
display(df.head())

print("\nℹ️ Información de tipos de datos:")
print(df.info())

print("\n❗ Porcentaje de valores faltantes por columna:")
missing_pct = df.isna().mean().sort_values(ascending=False) * 100
display(missing_pct)

# Distribución de la variable objetivo
if 'churn' not in df.columns:
    raise ValueError("La columna 'churn' no existe en el dataset. Verifica el archivo de entrada.")

churn_dist = df['churn'].value_counts(normalize=True).rename('proportion')
print("\n📊 Distribución de churn (0=se queda, 1=se va):")
display(churn_dist.to_frame())

churn_rate = churn_dist.get(1, 0)
print(f"➡️ Tasa global de churn en la muestra: {churn_rate:.3%}")


## 3️⃣ Calidad de datos: duplicados, faltantes y consistencia

Antes de modelar, debemos asegurar una base de calidad mínima:

1. **Duplicados de cliente**  
   - En entornos reales, es común tener duplicados por errores de integración.
   - Como regla, nos quedamos con **la primera ocurrencia** por `customer_id`.

2. **Valores faltantes**  
   - Variables numéricas: imputamos con **mediana** (robusto a outliers).
   - Variables categóricas: imputamos con la **moda** (valor más frecuente).

3. **Consistencia básica**  
   - No debería haber cargos negativos.
   - El número de servicios debe ser mayor o igual a 1.

Además, guardaremos algunos indicadores de calidad para documentar el estado del dato.


In [ ]:
# 3️⃣ Código – Limpieza de duplicados y valores faltantes

# Comprobación de duplicados por customer_id
if 'customer_id' in df.columns:
    dup_count = df['customer_id'].duplicated().sum()
    print(f"🧬 Registros duplicados por 'customer_id': {dup_count:,}")
    if dup_count > 0:
        df = df.drop_duplicates(subset='customer_id', keep='first')
        print(f"✅ Se eliminaron duplicados. Nuevas filas: {len(df):,}")
else:
    print("⚠️ No existe columna 'customer_id'. Se asume que cada fila es un cliente único.")

# Identificar columnas numéricas y categóricas (excluyendo target)
target_col = 'churn'
numeric_cols = df.select_dtypes(include=['number']).columns.drop(target_col)
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

print("\n📌 Columnas numéricas:", list(numeric_cols))
print("📌 Columnas categóricas:", list(categorical_cols))

# Imputación de valores faltantes
df_clean = df.copy()

# Numéricos → mediana
for col in numeric_cols:
    median_val = df_clean[col].median()
    missing_before = df_clean[col].isna().sum()
    df_clean[col].fillna(median_val, inplace=True)
    if missing_before > 0:
        print(f"✅ Imputados {missing_before} valores faltantes en '{col}' con mediana={median_val:.2f}")

# Categóricos → moda
for col in categorical_cols:
    mode_val = df_clean[col].mode().iloc[0]
    missing_before = df_clean[col].isna().sum()
    df_clean[col].fillna(mode_val, inplace=True)
    if missing_before > 0:
        print(f"✅ Imputados {missing_before} valores faltantes en '{col}' con moda='{mode_val}'")

print("\n✅ Porcentaje de valores faltantes después de limpieza:")
display(df_clean.isna().mean().sort_values(ascending=False) * 100)

# Reglas de consistencia simples
if 'monthly_charges' in df_clean.columns:
    negativos = (df_clean['monthly_charges'] < 0).sum()
    if negativos > 0:
        print(f"⚠️ Se encontraron {negativos} monthly_charges negativos. Se pasarán a NaN y se imputarán con la mediana.")
        df_clean.loc[df_clean['monthly_charges'] < 0, 'monthly_charges'] = np.nan
        median_val = df_clean['monthly_charges'].median()
        df_clean['monthly_charges'].fillna(median_val, inplace=True)


## 4️⃣ Exploración descriptiva: tamaño del problema y estructura de la base

En esta sección:

- Cuantificamos el churn por segmentos clave:
  - Tipo de contrato.
  - Segmento de cliente.
  - Método de pago.
- Observamos la distribución de variables numéricas relevantes:
  - Antigüedad.
  - Cargos mensuales.
  - Tickets y pagos tardíos.

El objetivo es entender **dónde se concentra el problema** antes de entrenar ningún modelo.


In [ ]:
# 4️⃣ Código – EDA descriptiva

df_eda = df_clean.copy()

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

if 'tenure_months' in df_eda.columns:
    axes[0, 0].hist(df_eda['tenure_months'], bins=30)
    axes[0, 0].set_title('Distribución de antigüedad (tenure_months)')
    axes[0, 0].set_xlabel('Meses')
    axes[0, 0].set_ylabel('Frecuencia')

if 'monthly_charges' in df_eda.columns:
    axes[0, 1].hist(df_eda['monthly_charges'], bins=30)
    axes[0, 1].set_title('Distribución de cargos mensuales')
    axes[0, 1].set_xlabel('Monto mensual')
    axes[0, 1].set_ylabel('Frecuencia')

if 'num_tickets_6m' in df_eda.columns:
    axes[1, 0].hist(df_eda['num_tickets_6m'], bins=11)
    axes[1, 0].set_title('Tickets de soporte últimos 6 meses')
    axes[1, 0].set_xlabel('Número de tickets')
    axes[1, 0].set_ylabel('Frecuencia')

if 'late_payments_6m' in df_eda.columns:
    axes[1, 1].hist(df_eda['late_payments_6m'], bins=7)
    axes[1, 1].set_title('Pagos tardíos últimos 6 meses')
    axes[1, 1].set_xlabel('Número de pagos tardíos')
    axes[1, 1].set_ylabel('Frecuencia')

plt.tight_layout()
plt.show()

def churn_rate_by(col):
    if col not in df_eda.columns:
        print(f"⚠️ La columna '{col}' no existe en el dataset.")
        return None
    grp = df_eda.groupby(col)['churn'].mean().sort_values(ascending=False)
    print(f"\n📊 Tasa de churn por '{col}':")
    display(grp.to_frame('churn_rate'))
    plt.figure(figsize=(7, 4))
    grp.plot(kind='bar')
    plt.title(f'Tasa de churn por {col}')
    plt.ylabel('Proporción de churn')
    plt.show()
    return grp

churn_by_contract = churn_rate_by('contract_type') if 'contract_type' in df_eda.columns else None
churn_by_segment = churn_rate_by('segment') if 'segment' in df_eda.columns else None
churn_by_payment = churn_rate_by('payment_method') if 'payment_method' in df_eda.columns else None


## 5️⃣ Feature engineering: enriquecer la visión del cliente

Para que el modelo capture mejor el comportamiento real, creamos variables derivadas:

- `tenure_years`: antigüedad en años.
- `is_new_customer`: indicador de clientes de **alta fragilidad** (≤ 6 meses).
- `avg_charge_per_service`: ticket promedio por servicio contratado.
- `has_many_tickets`: indicador de clientes con **alta fricción operativa**.
- `estimated_12m_revenue`: ingreso estimado de los próximos 12 meses *si el cliente se quedara*.

Estas variables permiten hablar de **riesgo económico**, no solo probabilidades abstractas.


In [ ]:
# 5️⃣ Código – Feature engineering

df_fe = df_clean.copy()

if 'tenure_months' in df_fe.columns:
    df_fe['tenure_years'] = df_fe['tenure_months'] / 12.0
    df_fe['is_new_customer'] = (df_fe['tenure_months'] <= 6).astype(int)
else:
    df_fe['tenure_years'] = np.nan
    df_fe['is_new_customer'] = 0

if 'num_services' in df_fe.columns:
    df_fe['num_services_clean'] = df_fe['num_services'].replace(0, 1)
else:
    df_fe['num_services_clean'] = 1

if 'monthly_charges' in df_fe.columns:
    df_fe['avg_charge_per_service'] = df_fe['monthly_charges'] / df_fe['num_services_clean']
    df_fe['estimated_12m_revenue'] = df_fe['monthly_charges'] * 12
else:
    df_fe['avg_charge_per_service'] = np.nan
    df_fe['estimated_12m_revenue'] = np.nan

if 'num_tickets_6m' in df_fe.columns:
    df_fe['has_many_tickets'] = (df_fe['num_tickets_6m'] >= 3).astype(int)
else:
    df_fe['has_many_tickets'] = 0

new_cols = ['tenure_years', 'is_new_customer', 'avg_charge_per_service',
            'estimated_12m_revenue', 'has_many_tickets']

print("✅ Columnas generadas:")
display(df_fe[new_cols].head())


## 6️⃣ Preparación para modelado: separación Train/Test y pipeline

En un entorno de producción es fundamental:

- Separar un conjunto de **entrenamiento** y otro de **prueba**.
- Evitar fugas de información (data leakage).
- Dejar el flujo listo para ser **empaquetado y versionado**.

En esta sección:

1. Definimos:
   - `X`: variables explicativas.
   - `y`: variable objetivo (`churn`).
2. Separamos Train/Test con estratificación en churn.
3. Definimos un **pipeline de sklearn** con:
   - `ColumnTransformer` para:
     - Dejar numéricos como están.
     - One-hot encoding de categóricos.
   - Modelo `RandomForestClassifier` como estimador principal.


In [ ]:
# 6️⃣ Código – Train/Test split y construcción de pipeline

TARGET_COL = 'churn'
ID_COL = 'customer_id' if 'customer_id' in df_fe.columns else None

if TARGET_COL not in df_fe.columns:
    raise ValueError("No se encuentra la columna 'churn' en el dataset.")

y = df_fe[TARGET_COL]

feature_cols = df_fe.columns.drop([TARGET_COL] + ([ID_COL] if ID_COL else []))
X = df_fe[feature_cols]

print(f"📐 X shape: {X.shape}, y length: {len(y)}")

numeric_features = X.select_dtypes(include=['number']).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

print("\n📌 Features numéricos:", numeric_features)
print("📌 Features categóricos:", categorical_features)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

print(f"\n🔀 Train: {X_train.shape[0]:,} filas  |  Test: {X_test.shape[0]:,} filas")

numeric_transformer = 'passthrough'
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, numeric_features),
        ('categorical', categorical_transformer, categorical_features)
    ]
)

rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=8,
    random_state=42,
    class_weight='balanced_subsample',
    n_jobs=-1
)

model_pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('model', rf_model)
    ]
)

print("\n✅ Pipeline preparado (preprocesamiento + modelo).")


## 7️⃣ Línea base (baseline): modelo Dummy

Antes de dar por bueno cualquier modelo, medimos una **línea base**:

- Usamos un `DummyClassifier` con estrategia **estratificada**.
- Este modelo “predice al azar” respetando la proporción de churn/no churn.

La regla es simple:

> **Si tu modelo no supera al Dummy, no tienes modelo.**

Evaluaremos:

- Accuracy
- Precision (churn=1)
- Recall (churn=1)
- F1
- ROC-AUC


In [ ]:
# 7️⃣ Código – Baseline Dummy

dummy_clf = DummyClassifier(strategy='stratified', random_state=42)
dummy_clf.fit(X_train, y_train)

y_pred_dummy = dummy_clf.predict(X_test)
y_proba_dummy = dummy_clf.predict_proba(X_test)[:, 1]

def evaluate_model(y_true, y_pred, y_proba, name='modelo'):
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, zero_division=0),
        'recall': recall_score(y_true, y_pred, zero_division=0),
        'f1': f1_score(y_true, y_pred, zero_division=0),
        'roc_auc': roc_auc_score(y_true, y_proba)
    }
    print(f"\n📊 Métricas para {name}:")
    for k, v in metrics.items():
        print(f"  {k:10s}: {v:.3f}")
    return metrics

metrics_dummy = evaluate_model(y_test, y_pred_dummy, y_proba_dummy, name='Dummy (baseline)')


## 8️⃣ Entrenamiento del modelo de churn (Random Forest)

Ahora entrenamos el modelo real:

- `RandomForestClassifier` encapsulado en el **pipeline** (preprocesa + entrena).
- Se entrena únicamente con el set de entrenamiento.
- Se evalúa sobre el set de prueba y se comparan las métricas contra el Dummy.

Buscamos:

- Mejor F1 y ROC-AUC que la baseline.
- Un Recall razonable para churn, para no dejar escapar demasiados clientes que realmente se van.


In [ ]:
# 8️⃣ Código – Entrenamiento y evaluación del modelo principal

model_pipeline.fit(X_train, y_train)

y_pred_rf = model_pipeline.predict(X_test)
y_proba_rf = model_pipeline.predict_proba(X_test)[:, 1]

metrics_rf = evaluate_model(y_test, y_pred_rf, y_proba_rf, name='Random Forest (pipeline)')

comparison = pd.DataFrame({
    'metric': ['accuracy', 'precision', 'recall', 'f1', 'roc_auc'],
    'dummy': [
        metrics_dummy['accuracy'],
        metrics_dummy['precision'],
        metrics_dummy['recall'],
        metrics_dummy['f1'],
        metrics_dummy['roc_auc'],
    ],
    'random_forest': [
        metrics_rf['accuracy'],
        metrics_rf['precision'],
        metrics_rf['recall'],
        metrics_rf['f1'],
        metrics_rf['roc_auc'],
    ],
})

print("\n📊 Comparativa Dummy vs Random Forest:")
display(comparison)


## 9️⃣ Matriz de confusión y diagnóstico de errores

Además de las métricas agregadas, es clave entender **cómo se equivoca el modelo**:

- ¿Cuántos clientes que se iban no se detectan? (falsos negativos).
- ¿Cuántos clientes se marcan erróneamente como en riesgo? (falsos positivos).

Mostraremos la **matriz de confusión** en valores absolutos y normalizados.


In [ ]:
# 9️⃣ Código – Matriz de confusión

def plot_confusion_matrix_custom(y_true, y_pred, labels=(0, 1)):
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_norm = cm / cm.sum(axis=1, keepdims=True)

    fig, axes = plt.subplots(1, 2, figsize=(10, 4))

    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0])
    axes[0].set_title('Matriz de confusión (absoluta)')
    axes[0].set_xlabel('Predicción')
    axes[0].set_ylabel('Real')

    sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Blues', ax=axes[1])
    axes[1].set_title('Matriz de confusión (normalizada)')
    axes[1].set_xlabel('Predicción')
    axes[1].set_ylabel('Real')

    plt.tight_layout()
    plt.show()

plot_confusion_matrix_custom(y_test, y_pred_rf)


## 🔟 Interpretabilidad: variables clave que explican el churn

Un modelo es útil en negocio cuando además de predecir, **explica el fenómeno**.

En esta sección:

- Extraemos la **importancia de características** del Random Forest.
- Identificamos las Top N variables que más peso tienen en la decisión.
- Relacionamos estas variables con acciones de negocio:
  - Tipo de contrato.
  - Tickets de soporte.
  - Pagos tardíos.
  - Antigüedad y descuentos.

Esto permite contestar preguntas como:

> “¿Qué palancas debemos mover para reducir el churn estructuralmente?”


In [ ]:
# 🔟 Código – Importancia de variables

rf_fitted = model_pipeline.named_steps['model']
preprocessor_fitted = model_pipeline.named_steps['preprocessor']

numeric_feature_names = numeric_features
cat_ohe = preprocessor_fitted.named_transformers_['categorical']
cat_feature_names = cat_ohe.get_feature_names_out(categorical_features)

all_feature_names = list(numeric_feature_names) + list(cat_feature_names)

feature_importances = pd.DataFrame({
    'feature': all_feature_names,
    'importance': rf_fitted.feature_importances_
}).sort_values(by='importance', ascending=False)

top_n = 20
print(f"🏆 Top {top_n} variables más importantes:")
display(feature_importances.head(top_n))

plt.figure(figsize=(8, 8))
sns.barplot(
    x='importance',
    y='feature',
    data=feature_importances.head(top_n)
)
plt.title(f'Top {top_n} características más importantes')
plt.xlabel('Importancia')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()


## 1️⃣1️⃣ Ranking de clientes por riesgo y simulación de campaña de retención

Con el modelo entrenado podemos construir un **score de riesgo de churn por cliente**:

1. Calculamos la probabilidad de churn para cada cliente en el conjunto de test.
2. Ordenamos de mayor a menor riesgo.
3. Definimos un segmento prioritario, por ejemplo:
   - Top 10% de clientes por probabilidad de churn.
4. Medimos:
   - Tasa de churn real dentro de ese top.
   - `Lift` respecto a la tasa global.
   - Ingreso estimado en riesgo (usando `estimated_12m_revenue`).

Esto permite estimar el **impacto económico** de una campaña focalizada.


In [ ]:
# 1️⃣1️⃣ Código – Ranking y simulación de campaña

proba_test = model_pipeline.predict_proba(X_test)[:, 1]

df_test_business = df_fe.loc[X_test.index].copy()
df_test_business['churn_real'] = y_test
df_test_business['prob_churn_model'] = proba_test

df_ranked = df_test_business.sort_values(by='prob_churn_model', ascending=False)

top_pct = 0.10
top_n_clients = int(len(df_ranked) * top_pct)
df_top = df_ranked.head(top_n_clients)

global_churn_rate = df_ranked['churn_real'].mean()
top_churn_rate = df_top['churn_real'].mean()
lift_top = top_churn_rate / global_churn_rate if global_churn_rate > 0 else np.nan

revenue_col = 'estimated_12m_revenue'
if revenue_col in df_top.columns:
    total_revenue_risk = df_top.loc[df_top['churn_real'] == 1, revenue_col].sum()
else:
    total_revenue_risk = np.nan

print(f"🎯 Tamaño conjunto Test: {len(df_ranked):,} clientes")
print(f"🎯 Top {top_pct:.0%} de riesgo: {top_n_clients:,} clientes\n")

print(f"📈 Tasa de churn global en Test:      {global_churn_rate:.2%}")
print(f"🔥 Tasa de churn en Top {top_pct:.0%}: {top_churn_rate:.2%}")
print(f"💡 Lift (Top vs Global):              {lift_top:.2f}x")

if not np.isnan(total_revenue_risk):
    print(f"\n💰 Ingreso anual estimado en riesgo dentro del Top (clientes que efectivamente se fueron):")
    print(f"   ≈ {total_revenue_risk:,.0f} unidades monetarias")

print("\n👀 Muestra de clientes priorizados (Top 10):")
cols_show = [c for c in ['customer_id', 'segment', 'contract_type', 'monthly_charges',
                         'tenure_months', 'num_tickets_6m', 'late_payments_6m',
                         'has_discount', 'churn_real', 'prob_churn_model'] if c in df_top.columns]

display(df_top[cols_show].head(10))


## 1️⃣2️⃣ Conclusiones ejecutivas y siguientes pasos

En esta sección sintetizamos los hallazgos en lenguaje de negocio.  
El objetivo es que un director pueda tomar decisiones sin ver el código.

El notebook imprime un **resumen ejecutivo dinámico** basado en los resultados obtenidos:

- Calidad del modelo comparado contra el baseline.
- Variables que explican el churn.
- Efectividad de priorizar el Top 10% de riesgo.
- Impacto económico potencial de actuar sobre ese segmento.

Además, se listan **siguientes pasos recomendados** para evolucionar la solución hacia producción:
- Industrialización del pipeline.
- Integración con CRM / sistema de campañas.
- Monitoreo de deriva del modelo (drift).


In [ ]:
# 1️⃣2️⃣ Código – Resumen ejecutivo dinámico

print("📌 RESUMEN EJECUTIVO DEL MODELO DE CHURN\n")

print("1) Desempeño del modelo vs baseline\n------------------------------------")
for metric in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']:
    base_val = metrics_dummy[metric]
    rf_val = metrics_rf[metric]
    delta = rf_val - base_val
    print(f"- {metric.upper():9s} | Dummy: {base_val:.3f} | RF: {rf_val:.3f} | Δ: {delta:+.3f}")

print("\n2) Drivers principales de churn\n--------------------------------")
for i, row in feature_importances.head(10).iterrows():
    print(f"- {row['feature']}: importancia {row['importance']:.3f}")

print("\n3) Efectividad de la priorización\n----------------------------------")
print(f"- Tasa de churn global en Test:      {global_churn_rate:.2%}")
print(f"- Tasa de churn en Top 10% riesgo:   {top_churn_rate:.2%}")
print(f"- Lift (Top vs global):              {lift_top:.2f}x")

if not np.isnan(total_revenue_risk):
    print(f"- Ingreso anual estimado en riesgo dentro del Top (clientes que efectivamente se fueron):")
    print(f"  ≈ {total_revenue_risk:,.0f} unidades monetarias")

print("\n4) Recomendaciones accionables\n--------------------------------")
print("- Implementar una campaña de retención específicamente dirigida al Top 10–20% de clientes por riesgo,")
print("  combinando beneficios económicos (descuentos, upgrades) con acciones de servicio (llamadas proactivas).")
print("- Diseñar iniciativas estructurales para reducir la fricción en los clientes con muchos tickets de soporte.")
print("- Incentivar, mediante oferta comercial, el paso de contratos 'Month-to-month' a contratos de mayor plazo.")
print("- Revisar políticas de cobro y recordatorio a clientes con alta incidencia de pagos tardíos.")
print("- Integrar este modelo al CRM para que el equipo de Customer Success trabaje con una vista priorizada de cartera.")
print("\n✅ Este notebook demuestra una solución de ciencia de datos lista para ser discutida con negocio y escalada a producción.")
